# Interpolate elevation data from the spline fit
(combine this with 6 since we cleaned up the code)

In [ ]:
from shapely.ops import nearest_points, LineString, Point
from pathlib import Path
import geopandas as gpd
import rasterio
from scipy.interpolate import splrep, splev, BSpline
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import json

from bikewaysim.paths import config
from bikewaysim.network import elevation_tools

In [ ]:
#for storing the interpolated points with sampled elevation data
import pickle
with (config['network_fp']/'spline_fit_elevation.pkl').open('rb') as fh:
    interpolated_points_dict = pickle.load(fh)

links = gpd.read_file(config['osmdwnld_fp']/'osm.gpkg',layer='edges')[['linkid','osmid','start_dist','end_dist','geometry']]
links['start_dist'] = (links['start_dist'] / 3.28084).round(2) # convert to meters
links['end_dist'] = (links['end_dist'] / 3.28084).round(2)

#cross reference
links = links[links['osmid'].isin(interpolated_points_dict.keys())]

#reproject network to DEM crs
with (config['network_fp']/'dem_crs.txt').open('r') as fh:
    dem_crs = fh.read()
links.to_crs(dem_crs,inplace=True)

In [ ]:
links[links['osmid']==155487818].explore()

Examine some spline fits

In [ ]:
test = interpolated_points_dict[155487818]

test['spline']
distance = test['distances']
elevation = test['elevations']
distance0 = np.array(list(range(0,distance[-1]+1,1)))
# smoothed_elevations = splev(distance,test['spline'])
smoothed_elevations = np.interp(distance0, distance, elevation)

# Create the plot
fig, ax = plt.subplots()

# Plot the elevation profile
ax.plot(distance, elevation, label='Elevation profile')
ax.plot(distance0, smoothed_elevations, label='Smoothed Elevations')

# Set labels
ax.set_xlabel('Distance (m)')
ax.set_ylabel('Elevation (m)')

# Ensure no vertical exaggeration by setting equal scale
# Here, the aspect ratio is based on the maximum range of the data
aspect_ratio = (max(distance) - min(distance)) / (max(elevation) - min(elevation))
ax.set_aspect(aspect_ratio)

# Add a grid and legend
ax.grid(True)
ax.legend()

# Show the plot
plt.show()
# elevation_tools.elevation_stats(distance,elevation,15)

Example

In [ ]:
# #select a link and try it
# linkid = 637636161
# link = links[links['osmid']==linkid].iloc[[0],:]

# #get osm line
# line = raw_links.loc[linkid,'geometry']#interpolated_points_dict[linkid]['geometry']
# line = np.array(line.coords)

# #get geo of start and end
# #or just use the included line to reduce memory?
# pointA = nodes[nodes['osm_N']==link['osm_A'].item()]
# pointB = nodes[nodes['osm_N']==link['osm_B'].item()]
# print(line)

In [ ]:
# # Define the coordinates of two additional points
# point1 = (pointA.geometry.item().x,pointA.geometry.item().y)
# point2 = (pointB.geometry.item().x,pointB.geometry.item().y)
# print(point1,point2)

In [ ]:
# # Plot the GeoDataFrame and the additional points
# fig, ax = plt.subplots()

# # plot the full link
# ax.plot(line[:,0],line[:,1], color='gray', label='full osm')

# link.plot(ax=ax, color='blue', label='osm segemnt')
# ax.plot(point1[0], point1[1], marker='o', color='red', markersize=10, label='Point 1')
# ax.plot(point2[0], point2[1], marker='o', color='green', markersize=10, label='Point 2')

# # Add labels to the additional points
# #ax.text(point1[0], point1[1], 'Point 1', fontsize=12, ha='right')
# #ax.text(point2[0], point2[1], 'Point 2', fontsize=12, ha='right')

# # Add legend and labels
# ax.legend()
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')
# ax.set_title('GeoDataFrame with Additional Points')

# # Manually set limits to create a square aspect ratio
# min_x, max_x = ax.get_xlim()
# min_y, max_y = ax.get_ylim()
# width = max(max_x - min_x, max_y - min_y)
# center_x = (min_x + max_x) / 2
# center_y = (min_y + max_y) / 2
# ax.set_xlim(center_x - width / 2, center_x + width / 2)
# ax.set_ylim(center_y - width / 2, center_y + width / 2)

# plt.show()


In [ ]:
# point1_geo = Point(point1)
# point2_geo = Point(point2)
# line_geo = LineString(line)

Find the distance of the shapepoint on each line

In [ ]:
# from shapely import line_locate_point, equals_exact

# point1_dist = line_locate_point(LineString(line),Point(point1))
# point2_dist = line_locate_point(LineString(line),Point(point2))

# #scenario 1: last point intersects with early point on a line (line loops into itself)
# #so trim off the points before point 1
# if point1_dist >= point2_dist:
#     for first_i, point in enumerate(line):
#         if equals_exact(Point(point),Point(point1),tolerance=1):
#             break
#     new_line = line[first_i+1:]
#     point2_dist = line_locate_point(LineString(new_line),Point(point1))

# #scenario 2: first point intersect with last point on a line
# #so trim off the point at the end of the line
# if point1_dist >= point2_dist:
#     new_line = line[0:-1]
#     point1_dist = line_locate_point(LineString(new_line),Point(point1))
#     point2_dist = line_locate_point(LineString(line),Point(point1))
    
# if point1_dist >= point2_dist:
#     print('error')
# else:
#     print(np.round(point1_dist),np.round(point2_dist))

# Interpolate distance on line
For each network link, use the pre-calculated start_dist and end_dist to interpolate elevation data from the fitted spline.

In [ ]:
# this is a more complicated case that we won't bother with
links[links['start_dist']>links['end_dist']].shape[0]

In [ ]:
# c = []
# from tqdm import tqdm
# for idx, row in tqdm(links.iterrows(),total=links.shape[0]):
#     osmid = row['osmid']
#     item = interpolated_points_dict.get(osmid,False)
#     #retrieve the fitted spline if it exists
#     spline = item.get('spline',0)
#     if isinstance(spline,int):
#         c.append(idx)
# links.loc[c[2300]]
# interpolated_points_dict.get(560689146,False)

In [ ]:
#BUG make sure we are not extrapolating
import math

result = {}
test = []
for idx, row in tqdm(links.iterrows(),total=links.shape[0]):
    osmid = row['osmid']
    start_dist = row['start_dist']
    end_dist = row['end_dist']
    segment_length = row['geometry'].length # no need to convert to meters here

    if start_dist > end_dist:
        continue

    #get osm distances and elevation
    item = interpolated_points_dict.get(osmid,False)
    if isinstance(item,bool):
        print('no record for',idx)
        continue
    
    #get the max osm_dist
    osm_dist = item['distances'][-1]

    #retrieve the fitted spline if it exists
    spline = item.get('spline',0)

    if isinstance(spline,int):
        # # in these cases a spline wasn't fit to the data because there were
        # # only three data points so just assign it the same data as the osm way
        xs = item['distances']
        # cond0 = (xs >= start_dist) & (xs <= end_dist)
        # xs = xs[cond0]
        ys = item['elevations']
        if np.isnan(ys).any():
            continue
            # test.append(idx)
        # ys = item['elevations'][cond0]
        # #drop na
        # cond1 = np.isnan(ys) == False
        # xs = xs[cond1]
        # ys = ys[cond1]
        # if len(xs) <= 1:
        #     test.append(idx)
        #     continue
        result[row['linkid']] = {'distances':xs,'elevations':ys}
        continue

    # normal case
    if start_dist < end_dist:
        #if the end dist is past the osm dist, use the osm dist
        #this can happen if there's a rounding error
        if end_dist > osm_dist:
            end_dist = osm_dist
        
        # if the segment is less than 10 m, just use the first and last point for grade estimation
        if segment_length < 10:
            xs = [start_dist,end_dist]
            ys = splev(xs,spline)
        else:
            #get x values from start_dist to end_dist every 10 m
            xs = []
            while start_dist < end_dist:
                xs.append(start_dist)
                start_dist += 10
        
        xs = np.array(xs)
        # ys = splev(xs,spline)
        #NOTE I was getting a spline fit that was giving ludicrous numbers when interpolating
        # until I can fix that, using piecewise linear interpolation which should be okay
        ys = np.interp(xs, item['distances'], item['elevations'])
        #normalize xs so that it goes from 0 to end for the elevation stats function
        xs = xs - xs[0]
        result[row['linkid']] = {'distances':np.array(xs),'elevations':ys}
    
    # TODO applies to so few cases that i don't think it makes a lot of sense to bother just leave as na
    # elif start_dist > end_dist:
    #     # if segment is less than 10 meters
    #     if math.abs(start_dist - end_dist) < 10:
    #         xs0 = [0,end_dist]
    #         ys0 = splev(xs0,spline)
    #         xs1 = [start_dist,osm_dist]
    #         ys1 = splev(xs1,spline)
    #         result[row['linkid']] = {'distances':np.array(xs1+xs0),'elevations':np.hstack([segment_1_ys,segment_2_ys])}
    #         continue
        
    #     # first segment goes from 0 m to end node
    #     segment_1_dist = 0
    #     segment_1 = []
    #     while segment_1_dist < end_dist:
    #         segment_1.append(segment_1_dist)
    #         segment_1_dist += 10

    #     # second segment goes from start node to end of osm way
    #     segment_2_dist = start_dist
    #     segment_2 = [segment_2_dist]
    #     while segment_2_dist < osm_dist:
    #         segment_2.append(segment_2_dist)
    #         segment_2_dist += 10

    #     xs = segment_2+segment_1
    #     ys = splev(xs,spline) 

In [ ]:
# len(test)
# interpolated_points_dict[links.loc[test[40],'osmid']]
# links.loc[[test[1]]].explore()

In [ ]:
# import random
# links.loc[test].explore()

In [ ]:
# reload(elevation_tools)
# export into QGIS to examine
export = {linkid:elevation_tools.simple_elevation_stats(item['distances'],item['elevations']) for linkid, item in result.items()}
export = pd.DataFrame.from_dict(export,orient='index')

df = pd.merge(links,export,left_on='linkid',right_index=True)
df['ascent_ft'] = (df['ascent_m'] * 3.28084).round(0)
df['descent_ft'] = (df['descent_m'] * 3.28084).round(0)
df['max_elev'] = np.max(np.abs(df[['ascent_ft','descent_ft']].values),axis=1)
df['max_grade'] = np.max(np.abs(df[['ascent_grade_%','descent_grade_%']].values),axis=1)
df['descent_ft'] = df['descent_ft'].abs()
df['descent_grade_%'] = df['descent_grade_%'].abs()
gdf = gpd.GeoDataFrame(df,crs=dem_crs)
gdf.to_crs(config['projected_crs_epsg'],inplace=True)
print(gdf['max_grade'].isna().sum(),'nan grades')
#remove nan grades, we'll fill them in the next one
gdf = gdf[gdf['max_grade'].notna()]

#fill in any nan values


# turn into 3 categories [< 4, 4-6, > 6]
bins = [0,4,6,np.inf]
labels = ['[0,4)','[4,6)','[6,inf)']
gdf['ascent_grade_cat'] = pd.cut(gdf['ascent_grade_%'],bins=bins,labels=labels,include_lowest=True).astype(str)
gdf['descent_grade_cat'] = pd.cut(gdf['descent_grade_%'],bins=bins,labels=labels,include_lowest=True).astype(str)
gdf['max_grade_cat'] = pd.cut(gdf['max_grade'],bins=bins,labels=labels,include_lowest=True).astype(str)

In [ ]:
x = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_links',ignore_geometry=True)[['osm_linkid','link_type']]
x = x.set_index('osm_linkid')['link_type'].to_dict()
gdf['link_type'] = gdf['linkid'].map(x)

# Export

In [ ]:
gdf.to_file(config['network_fp']/'elevation.gpkg')

In [ ]:
gdf.loc[gdf['max_grade'].idxmax()]

In [ ]:
interpolated_points_dict[414497285]['distances']

In [ ]:
# row = links.loc[62421].squeeze()
# idx = 62421

# osmid = row['osmid']
# start_dist = row['start_dist']
# end_dist = row['end_dist']
# segment_length = row['geometry'].length # no need to convert to meters here

# # if start_dist > end_dist:
# #     continue

# #get osm distances and elevation
# item = interpolated_points_dict.get(osmid,False)
# # if isinstance(item,bool):
# #     print('no record for',idx)
# #     continue

# #get the max osm_dist
# osm_dist = item['distances'][-1]

# #retrieve the fitted spline if it exists
# spline = item.get('spline',0)

# if isinstance(spline,int):
#     # # in these cases a spline wasn't fit to the data because there were
#     # # only three data points so just assign it the same data as the osm way
#     xs = item['distances']
#     # cond0 = (xs >= start_dist) & (xs <= end_dist)
#     # xs = xs[cond0]
#     ys = item['elevations']
#     # if np.isnan(ys).any():
#         # continue
#         # test.append(idx)
#     # ys = item['elevations'][cond0]
#     # #drop na
#     # cond1 = np.isnan(ys) == False
#     # xs = xs[cond1]
#     # ys = ys[cond1]
#     # if len(xs) <= 1:
#     #     test.append(idx)
#     #     continue
#     result[row['linkid']] = {'distances':xs,'elevations':ys}
#     # continue

# # normal case
# if start_dist < end_dist:
#     #if the end dist is past the osm dist, use the osm dist
#     #this can happen if there's a rounding error
#     if end_dist > osm_dist:
#         end_dist = osm_dist
    
#     # if the segment is less than 10 m, just use the first and last point for grade estimation
#     if segment_length < 10:
#         xs = [start_dist,end_dist]
#         ys = splev(xs,spline)
#     else:
#         #get x values from start_dist to end_dist every 10 m
#         xs = []
#         i = start_dist
#         while i < end_dist:
#             xs.append(i)
#             i += 10
    
#     xs = np.array(xs)
#     ys = splev(xs,spline)
#     #normalize xs so that it goes from 0 to end for the elevation stats function
#     # xs = xs - xs[0]
#     #result[row['linkid']] = {'distances':np.array(xs),'elevations':ys}
# xs
# ys
# # spits out high values?
# splev(601,spline)

# Move on

In [ ]:
gdf.to_file(Path.home()/'Downloads/scratch.gpkg',layer='network_grade')

In [ ]:
gdf[gdf['max_grade'].isna()].explore()

In [ ]:
result[13803]

In [ ]:
interpolated_points_dict[176229031]['spline']

In [ ]:
# gdf.sort_values('max_elev').tail(20).explore()


In [ ]:
gdf.shape[0]

In [ ]:
gdf[gdf['max_grade']>30].explore()#.shape[0]
# gdf[gdf['max_grade']>15].explore()

In [ ]:
gdf['max_grade']

In [ ]:
gdf.sort_values('max_grade').tail(20)

In [ ]:
gdf.loc[gdf['linkid']==62421]

In [ ]:
item = result[62421]
item['distances']

In [ ]:
interpolated_points_dict[414497285]['distances']

In [ ]:
item['elevations']

In [ ]:
elevation_tools.elevation_stats(item['distances'],item['elevations'],5) 

In [ ]:


interpolate_dist_m = 1
new_elevations_dict = {}
spline_or_nah = []

for idx, row in interpolated.iterrows():
    #get ids
    osm_linkid = row['osm_linkid']
    osmid = row['osmid']

    #get osm elevation
    item = interpolated_points_dict[osmid]

    #retrieve the fitted spline if it exists
    spline = item.get('spline',0)
    
    #get segments
    segment = row['segment']
    
    #if no spline, do linear interpolatation
    if spline == 0:
        new_xs = item['distances']
        new_elevations = item['elevations']
        spline_or_nah.append(idx)

    elif isinstance(segment[0],list):
        #get x sequence
        new_xs_segment1 = np.arange(int(segment[0][0]),int(segment[0][1])+interpolate_dist_m,interpolate_dist_m)
        new_xs_segment2 = np.arange(int(segment[1][0]),int(segment[1][1])+interpolate_dist_m,interpolate_dist_m)

        #get new elevations
        new_elevations_segment1 = splev(new_xs_segment1,spline)
        new_elevations_segment2 = splev(new_xs_segment2,spline)

        #combine
        new_xs = np.arange(0,len(new_xs_segment1)+len(new_xs_segment2))
        new_elevations = np.hstack([new_elevations_segment2,new_elevations_segment1])

    else:
        #get new elevation values
        new_xs = np.arange(int(segment[0]),int(segment[1])+interpolate_dist_m,interpolate_dist_m)
        new_elevations = splev(new_xs, spline)
        #recalculate elevations stats
        #TODO if wanting to seperate grades by section use the elevation_stats function
    
    new_elevations_dict[idx] = elevation_tools.simple_elevation_stats(new_xs, new_elevations)


In [ ]:
from shapely import line_locate_point, equals_exact
from shapely.ops import polygonize, unary_union
from itertools import product

interpolated_distance_dict = {}

for idx, row in links.iterrows():
    
    #network link attributes
    osmid = row['osmid']
    a = row['osm_A']
    b = row['osm_B']
    network_line = np.array(row['geometry'].coords)
    point1 = network_line[0]
    point2 = network_line[-1]

    #get raw osm attributes
    line = np.array(raw_links.loc[osmid,'geometry'].coords)
    points = [Point(x) for x in line]
    points = gpd.GeoSeries(points)
    distances = points.distance(points.shift(1)).cumsum().tolist()
    distances[0] = 0
    node_list = json.loads(raw_links.loc[osmid,'all_tags'])['@way_nodes']
    a_idx = node_list.index(a)
    b_idx = node_list.index(b)

    #polygon check
    poly_check = len(list(polygonize(unary_union(LineString(line))))) #> 0
    poly_check_network = len(list(polygonize(unary_union(LineString(network_line))))) #> 0

    method = 'none'
    reverse_geometry = False
    
    # scenario 1: simple line segment no loops
    if (poly_check == 0) & (poly_check_network == 0) & (a != b):
        method = 'simple'
        # step 1: check if network line is in the same direction as loop
        if (a_idx > b_idx):     
            reverse_geometry = True
            a = row['osm_B']
            b = row['osm_A']
            a_idx = node_list.index(a)
            b_idx = node_list.index(b)

        # step 2: get distance of start and end point along the line
        #point1_dist = line_locate_point(LineString(line),Point(point1))
        #point2_dist = line_locate_point(LineString(line),Point(point2))
        network_dist = distances[a_idx:b_idx+1]
        point1_dist = network_dist[0]
        point2_dist = network_dist[-1]

        # step 3: check to see if distance matches the network geometry
        difference = point2_dist - point1_dist
        distance_check = np.abs(difference - LineString(network_line).length) < 0.02
        if distance_check == False:
            print('error')

        segment = [point1_dist,point2_dist]
    
    # scenario 2: network segment is not a loop but the line is
    elif (poly_check > 0) & (poly_check_network == 0) & (a != b):
        method = 'simple loop'

        a_s = [idx for idx, val in enumerate(node_list) if val == a]
        b_s = [idx for idx, val in enumerate(node_list) if val == b]

        #get all possible combinations of A/B
        combinations = list(product(a_s,b_s))
        for a_idx, b_idx in combinations:
            # step 1: see if network line direction matches line
            if (a_idx > b_idx):
                reverse_geometry = True
                # a = row['osm_B']
                # b = row['osm_A']
                # a_idx = node_list.index(a)
                # b_idx = node_list.index(b)
                a_idx, b_idx = b_idx, a_idx

            # step 2: get distance of start and end point along the line
            # start = node_list.index(a)
            # end = node_list[start+1:].index(b) + start + 1
            network_dist = distances[a_idx:b_idx+1]
            point1_dist = network_dist[0]
            point2_dist = network_dist[-1]

            # step 3: check to see if distance matches the network geometry
            difference = point2_dist - point1_dist
            distance_check = np.abs(difference - LineString(network_line).length) < 0.02
            
            segment = [point1_dist,point2_dist]

            if distance_check == True:
                break

        # step 4: if it doesn't then it is likely that the network segment crosses the loop
        # we'll need multiple segments in that case
        if distance_check == False:
            method = 'complex loop'

            #reset the reversed geometry bit
            reverse_geometry = False
            a = row['osm_A']
            b = row['osm_B']
            a_idx = node_list.index(a)
            b_idx = node_list.index(b)

            # if the network line crosses the loop then b_idx will be larger
            if a_idx < b_idx:
                reverse_geometry = True
                a = row['osm_B']
                b = row['osm_A']
                a_idx = node_list.index(a)
                b_idx = node_list.index(b)

            # step 5: get the two segments
            first_segment = [0,distances[b_idx]]
            second_segment = [distances[a_idx],distances[-1]]

            # step 6: check distance
            difference = second_segment[-1] - second_segment[0] + first_segment[1]
            distance_check = np.abs(difference - LineString(network_line).length) < 0.02

            # if distance_check == False:
            #     print('error')
            segment = [first_segment,second_segment]

    else:
        method = 'none'
        segment = 999
        distance_check = False
    
    #finaly assemble the dict
    interpolated_distance_dict[idx] = {
    'method': method,
    'segment': segment,
    'reverse_geometry': reverse_geometry,
    'distance_check': distance_check,
    'poly_check': poly_check
    }

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(interpolated_distance_dict,orient='index')
interpolated = pd.merge(links,df,left_index=True,right_index=True)

#drop loops
interpolated = interpolated[interpolated['method']!='none']

In [ ]:
interpolated

In [ ]:
#todo create a checkpoint here

In [ ]:
#export['segment'] = export['segment'].astype(str)
#export['length_m'] = export.length
#export.to_file(Path.home()/'Downloads/scratch.gpkg',layer='all')

# Spline fit
We need elevation data between the two points to interpolate an elevation profile for the smaller links

In [ ]:


interpolate_dist_m = 1
new_elevations_dict = {}
spline_or_nah = []

for idx, row in interpolated.iterrows():
    #get ids
    osm_linkid = row['osm_linkid']
    osmid = row['osmid']

    #get osm elevation
    item = interpolated_points_dict[osmid]

    #retrieve the fitted spline if it exists
    spline = item.get('spline',0)
    
    #get segments
    segment = row['segment']
    
    #if no spline, do linear interpolatation
    if spline == 0:
        new_xs = item['distances']
        new_elevations = item['elevations']
        spline_or_nah.append(idx)

    elif isinstance(segment[0],list):
        #get x sequence
        new_xs_segment1 = np.arange(int(segment[0][0]),int(segment[0][1])+interpolate_dist_m,interpolate_dist_m)
        new_xs_segment2 = np.arange(int(segment[1][0]),int(segment[1][1])+interpolate_dist_m,interpolate_dist_m)

        #get new elevations
        new_elevations_segment1 = splev(new_xs_segment1,spline)
        new_elevations_segment2 = splev(new_xs_segment2,spline)

        #combine
        new_xs = np.arange(0,len(new_xs_segment1)+len(new_xs_segment2))
        new_elevations = np.hstack([new_elevations_segment2,new_elevations_segment1])

    else:
        #get new elevation values
        new_xs = np.arange(int(segment[0]),int(segment[1])+interpolate_dist_m,interpolate_dist_m)
        new_elevations = splev(new_xs, spline)
        #recalculate elevations stats
        #TODO if wanting to seperate grades by section use the elevation_stats function
    
    new_elevations_dict[idx] = elevation_tools.simple_elevation_stats(new_xs, new_elevations)


In [ ]:
interpolated.loc[spline_or_nah,'spline_fit'] = False

In [ ]:
df = pd.DataFrame.from_dict(new_elevations_dict,orient='index')
elevations_added = pd.merge(interpolated,df,left_index=True,right_index=True)

In [ ]:
elevations_added['segment'] = elevations_added['segment'].astype(str)
elevations_added.to_file(config['network_fp']/'elevation.gpkg')

In [ ]:
elevations_added.head()

In [ ]:
elevations_added[['ascent_grade_%','descent_grade_%']].abs().max()

In [ ]:
(elevations_added[['ascent_grade_%','descent_grade_%']] > 5).sum()

In [ ]:
elevations_added[(elevations_added[['ascent_grade_%','descent_grade_%']] > 5).any(axis=1)]

In [ ]:
elevations_added[['ascent_grade_%','descent_grade_%']].abs().describe()

In [ ]:
# from scipy.interpolate import splrep, splev, BSpline

# # case 1: only one segment to deal with
# osm_linkid = 1125605250
# osmid = 751119047

# row = export[export['osm_linkid']==osm_linkid].squeeze()

# #get osm elevation
# item = interpolated_points_dict[osmid]

# #retrieve the fitted spline
# spline = item['spline']

# #get segments
# segment = row['segment']

# #get x sequence
# new_xs_segment1 = np.arange(int(segment[0][0]),int(segment[0][1])+interpolate_dist_m,interpolate_dist_m)
# new_xs_segment2 = np.arange(int(segment[1][0]),int(segment[1][1])+interpolate_dist_m,interpolate_dist_m)

# #get new elevations
# new_elevations_segment1 = splev(new_xs_segment1,spline)
# new_elevations_segment2 = splev(new_xs_segment2,spline)

# #combine
# new_xs = np.arange(0,len(new_xs_segment1)+len(new_xs_segment2))
# new_elevations = np.hstack([new_elevations_segment2,new_elevations_segment1])

# fig, ax = plt.subplots()
# ax.plot(item['distances'],item['elevations'],'-')
# ax.plot(new_xs_segment1,new_elevations_segment1,'-.')
# ax.plot(new_xs_segment2,new_elevations_segment2,'-.')

In [ ]:
# # use fitted spline to from dict to create new values

# item = interpolated_points_dict[osmid]
# spline = interpolated_points_dict[osmid]['spline']

# interpolate_dist_m = 1
# new_xs = np.arange(int(point1_dist),int(point2_dist)+interpolate_dist_m,interpolate_dist_m)
# new_elevations = splev(new_xs, spline)

#new_grades = pd.Series(new_elevations).diff() #/ interpolate_dist_m * 100

In [ ]:
# from importlib import reload
# import src.elevation_tools as elevation_tools
# reload(elevation_tools)
# elevation_tools.simple_elevation_stats(new_xs, new_elevations)
# fig, ax = plt.subplots()
# ax.plot(item['distances'],item['elevations'],'-')
# ax.plot(new_xs,new_elevations,'-.')

In [ ]:
# linkid
# item = interpolated_points_dict[linkid]
# import numpy as np
# from scipy.interpolate import splrep, splev, BSpline

# spline = interpolated_points_dict[linkid]['spline']

# new_xs = np.arange(int(point1_dist),int(point2_dist)+10,10)
# new_ys = splev(new_xs, spline)
# new_ys
# fig, ax = plt.subplots()
# ax.plot(item['distances'],item['elevations'],'-')
# ax.plot(new_xs,new_ys,'-.')

Add elevation data to links (deprecated)

In [ ]:
# grade_cats = {}

# for linkid, item in tqdm(interpolated_points_dict.items()):     

#     outputs = elevation_tools.elevation_stats(item['distances'],item['smoothed'],grade_threshold)

#     df = pd.DataFrame({
#         'distance_deltas':outputs['distance_deltas'],
#         'segment_grades':outputs['segment_grades']
#         })
#     #create bins (broach 2012 ones)
#     #'(-inf,-6]','(-6,-4]','(-4,-2]','(-2,0]',
#     #-np.inf,-6,-4,-2,
#     bins = [-1,2,4,6,10,15,np.inf]
#     names = ['(0,2]','(2,4]','(4,6]','(6,10]','(10,15]','(15,inf]']
#     df['grade_category'] = pd.cut(df['segment_grades'].abs(), bins, labels = names)

#     #determine if up or down (doesn't matter if flat)
#     df.loc[df['segment_grades'] >= 0,'ascent_or_descent'] = 'ascent'
#     df.loc[df['segment_grades'] < 0,'ascent_or_descent'] = 'descent'

#     # how many meters at each grade category?
#     test = df.groupby(['grade_category','ascent_or_descent'],observed=False)['distance_deltas'].sum().round(0)
#     test = pd.DataFrame(test).transpose()
#     test.index = [linkid]
#     grade_cats[linkid] = {
#         'ascent_m': outputs['ascent'],
#         'descent_m': outputs['descent'],
#         'ascent_grade': outputs['ascent_grade'],
#         'descent_grade': outputs['descent_grade']
#     }
#     grade_cats[linkid].update(
#         test.to_dict(orient='records')[0]
#     )
    
# #.items())#.reseorient='t_index()#.pivot(
#     # columns=['grade_category','up'],
#     # values='distance_deltas'
#     # )
# #grade_cats